#Installing the requirements

In [3]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf

In [ ]:
! pip install langchain

google-generativeai langchain-google-genai bothe of these were downloaded to load the geminin pro LLM. while chromadb is used for vector storage and pypdf is used to extract text from pdfs

#Importing libraries

In [5]:
import pandas as pd
import textwrap
from google.colab import userdata
from IPython.display import Markdown
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

#Setting up the models

In [6]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True)) # function for converting the LLM response into a clearly readable output

In [15]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY') # Setting up the google api key
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,temperature=0.2,convert_system_message_to_human=True)# importing gemini pro using langchain
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY) # choosing the embedding model

#Data cleaning and vectorizing

In [52]:
pdf_loader = PyPDFLoader("/content/Dish.pdf")
pages = pdf_loader.load_and_split()#Loading the pdf and splitting into pages

In [67]:
print(pages[3].page_content)

Taj
Beef
Burger:
Taj
Beef
Burger,
featuring
180g
ground
beef,
homemade
pickles,
lettuce,
tomato
&
cheddar
cheese
served
with
hand-cut
fries,
is
priced
at
Rs.169.
Add
Avocado:
Adding
Avocado
to
Taj
Beef
Burger
is
priced
at
Rs.15.
Taj
Chicken
Burger:
Taj
Chicken
Burger,
with
cajun
spiced
chicken,
cumin
raita,
rocket
&
tomato
served
with
hand-cut
fries,
is
priced
at
Rs.149.
Add
Avocado:
Adding
Avocado
to
Taj
Chicken
Burger
is
priced
at
Rs.15.
Chicken
Tikka
Masala:
Chicken
Tikka
Masala,
featuring
charred
chicken,
fenugreek
&
onion,
tomato
sauce,
is
priced
at
Rs.205.
Paneer
Sirka
Pyaaz:
Paneer
Sirka
Pyaaz,
a
cottage
cheese
curry
in
fresh
tomato
sauce
&
pickled
shallots,
is
priced
at
Rs.175.
Dal
Tadka:
Dal
Tadka,
featuring
yellow
lentils
tempered
with
cumin
&
asafoetida,
is
priced
at
Rs.145.
Dal
Makhni:
Dal
Makhni,
featuring
black
lentils
&
beans
simmered
overnight
in
an
authentic
recipe,
is
priced
at
Rs.145.
Karoo
Lamb
Chops:
Karoo
Lamb
Chops,
featuring
free-range
lamb
chops,
creamy
mash
po

In [54]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)
#joining the content in diffrent pages

In [65]:
texts

['Seared\nLine\nFish:\nThe\nSeared\nLine\nFish\ncontains\ncreamy\nmash\npotato,\nwilted\nspinach\n&\nlemon\nbutter\nand\ncosts\nRs.195.\nPrawns\nButterfly:\nPrawns\nButterfly\ncomes\nwith\nlemon\nbutter\nsauce,\nhand-cut\nfries\n&\nMediterranean\nsalad\nand\ncosts\nRs.235.\nKaran\nFillet:\nKaran\nFillet\nincludes\nfree-range\nbeef,\ncreamy\nmash\npotato,\nwilted\nspinach\n&\njus\nand\nis\npriced\nat\nRs.255.\nKaran\nSirloin:\nKaran\nSirloin\nfeatures\nfree-range\nbeef,\ncreamy\nmash\npotato,\nwilted\nspinach\n&\njus\nand\nis\npriced\nat\nRs.245.\nTandoori\nPlatter:\nTandoori\nPlatter\nconsists\nof\nmasala\nKaroo\nlamb\nchops,\nDelhi\nchicken\ntikka\n&\nbasil\ntulsi\nprawns\nand\ncosts\nRs.475.\nSteamed\nBasmati\nRice:\nSteamed\nBasmati\nRice\nis\npriced\nat\nRs.50.\nHand\nCut\nFries:\nHand\nCut\nFries\nare\npriced\nat\nRs.50.\nFrench\nFries:\nFrench\nFries\nare\npriced\nat\nRs.50.\nCreamed\nMash\nPotato:\nCreamed\nMash\nPotato\nis\npriced\nat\nRs.50.\nRocket\n&\nFeta:\nRocket\n&\nFeta\

In [56]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5}) # create a retreiver

#Outputs

In [57]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True
)
#form a QnA RAG chain using langchain

In [58]:
template = """Use the following pieces of context to answer the question at the end.
Here are examples of identifying price and ingredients :

- question : what is the price of Truffle Mushrooms?
answer : The price of Truffle Mushrooms is Rs.110

- question : what does Breakfast Smoothie comes with?
answer : It comes with muesli, banana, & honey

- question : what is the price of Bakery Basket?
answer : The price of Truffle Mushrooms is Rs.95

- question : what does Eggs Benedict, Florentine, or Royale comes with?
answer : It comes with with gypsy ham, spinach or smoked salmon, toasted muffin with hollandaise

- question : what does Breakfast Smoothie comes with?
answer : It comes with muesli, banana, & honey

If you don't know the answer, just say that you don't know; don't try to make up an answer. Keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
Also, before mentioning any price, prefix it with Rs.

Context:
{context}

Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [68]:
question = "Suggest a dish which is spicy"
result = qa_chain({"query": question})
Markdown(result["result"])

I'm sorry, but the context does not mention anything about spicy dishes, so I cannot answer this question.
Thanks for asking!

In [61]:
question = "what Warm Gulab Jamun comes with?"
result = qa_chain({"query": question})
Markdown(result["result"])

Warm Gulab Jamun comes with syrup-poached dumplings, vanilla anglaise, and crushed pistachio.
Thanks for asking!
